### <center>Assignment 4:
**Submitted by:** <br>
Sharanya Saha<br>
21111056 <br>
*sharanya21@iitk.ac.in*


In [1]:
#importing the required Libraries
import pandas as pd
import numpy as np
from random import shuffle

**Reading the datasets:**

In [2]:
column_names=['weight','height','label']
X=pd.read_csv('X.csv',names=column_names) #The training dataset
Y=pd.read_csv('y.csv',names=column_names[0:2]) #The test dataset

In [3]:
X.head()

,weight,height,label
0,48,58,1
1,54,62,1
2,48,56,1
3,46,62,1
4,47,59,1


In [4]:
Y

,weight,height
0,74,67
1,69,63
2,92,81
3,64,61
4,66,84
5,76,68
6,61,58
7,64,76
8,68,66
9,34,61


#### Answer 1: Implementation of Generalized Context Model 

Below is an Implementation of GCM from the lecture slides :

The function below takes the training set(X) and the distinct labels(category) as arguement and forms the exemplar sets. It returns the exemplar sets formed as a list of dataframes.

In [5]:
def form_exemplars(X,category=[1,2,3]) :
    exemplars=[]
    for i in category :
        df=X.loc[X['label'] == i]
        df=df[['height','weight']]
        exemplars.append(df)
    return exemplars

In [6]:
exemplars=form_exemplars(X)
len(exemplars)

3

The function below returns the count of the number of times x has been recorded for being in a particular category(exemplar set).

In [7]:
def get_count(x,exemplar_set) :
    count=exemplar_set.loc[(exemplar_set['height']==x['height']) & (exemplar_set['weight']==x['weight'])]
    return len(count)

The function below computes the psychological distance between memory exemplar x and stimulus y as: <br>
$$d(x,y)= \sum_{i} \alpha_{i}(x_{i}-y_{i})$$
where, $\alpha$ contains the attention weights (It reflects the role of semantic knowledge in categorization). <br>
As distance is always greater than zero, abs function is used outside the summation.

In [8]:
def distance(x,y,alpha) :
    dist=0
    for i in range(len(x)) :
        dist=dist+(alpha[i]*(x[i]-y[i]))
    return abs(dist)
    

The function below computes the similarity between memory exemplar x and stimulus y as :
$$ s(x,y)=exp(-\beta d(x,y))$$
d(x,y) is the distance between x and y (uses the function distance).<br> $\beta$ is a parameter reflecting bias-variance tradeoff in similarity judgements.

In [9]:
def similarity(x,y,alpha,beta) :
    return np.exp(-beta*distance(x,y,alpha))

The function below returns the category responses for a stimulus y. <br>
It uses the equation below where each exemplar vote for the category they are associated with.:


$$ p(R|y) = \frac{\gamma_{R}\sum_{x \in R} N(R,x)s(x,y)}{\sum_{r}\gamma_{r}\sum_{k \in r} N(R,k)s(k,y)}$$

$\gamma$ is the response bias parameter, reflecting the environmental priors on categorization. <br>
N(R,x) is the count of the number of times x has been recorded for being in category R before.


In [10]:
def compute_probability(y,exemplars,alpha,beta,gamma) :
    probability=[]
    j=0
    for exemplar_set in exemplars :
        category_votes=0
        for i in range(len(exemplar_set)) :
            x=exemplar_set.iloc[i]
            Nrx=get_count(x,exemplar_set)
            #print(Nrx)
            category_votes=category_votes+(Nrx*similarity(x,y,alpha,beta))
        #print(category_votes*gamma[j])
        probability.append(category_votes*gamma[j])
        j=j+1
    return probability/np.sum(probability)

Category Labels: small = 1, average = 2, large = 3<br>
- As the model is more likely to use weight than height to make the judgements, the attention weight for 'weight' should be greater than that of attention weight for 'height'. i.e $\alpha_{weight} > \alpha_{height}$
- As the model is more likely to call someone big average than large, $\gamma$ for category 2 should be greater than $\gamma$ for category 3.

In [11]:
alpha=[0.75,0.25] #attention weights
gamma=[0.2,0.7,0.3] #category 1 is at index 0, category 2 at index 1, category 3 at index 2
beta=1
Y_predicted=Y
Y_predicted['Predicted label']=[0]*len(Y)
for i in range(len(Y)) :
    probability=compute_probability(Y.iloc[i],exemplars,alpha,beta,gamma)
    result = np.where(probability == np.amax(probability))
    Y_predicted.at[i,'Predicted label']=result[0][0]+1

In [12]:
Y_predicted

,weight,height,Predicted label
0,74,67,2
1,69,63,2
2,92,81,3
3,64,61,2
4,66,84,2
5,76,68,3
6,61,58,1
7,64,76,2
8,68,66,2
9,34,61,1


#### Answer 2: Using Rational Model of categorization on the given dataset

In [13]:

# Implementation of Anderson's venerable "rational" model of categorization.
# Assumes that stimuli were generated by a mixture of Gaussian distributions;
# rather than compute the full Bayesian posterior, it views items sequentially
# and assigns each to the maximum a posteriori cluster.
#
# At the end it is presented with a stimulus with one item missing, and
# predicts the probability that its value is a '0' or a '1'.
#
# Implemented in python by John McDonnell
#
# References: Anderson (1990) and Anderson (1991),



#Utility functions:

class dLocalMAP:
    """
    See Anderson (1990, 1991)
    'Categories' renamed 'clusters' to avoid confusion.
    Discrete version.
    
    Stimulus format is a list of integers from 0 to n-1 where n is the number
    of possible features (e.g. [1,0,1])
    
    args: c, alphas
    """
    
    def __init__(self, args):
        self.partition = [[]]
        self.c, self.alpha = args
        self.alpha0 = [sum(s) for s in self.alpha.T]
        #print(self.alpha0)
        self.N = 0
    
    def probClustVal(self, k, i, val):
        """Find P(j|k)"""
        cj = len([x for x in self.partition[k] if x[i]==val])
        nk = len(self.partition)
        #print('i',i)
        return (cj + self.alpha[i][val])/(nk + self.alpha0[i])
    
    def condclusterprob(self, stim, k):
        """Find P(F|k)"""
        pjks = []
        for i in range(len(stim)):
            cj = len([x for x in self.partition[k] if x[i]==stim[i]])
            nk = len(self.partition[k])
            #print('i',i)
            pjks.append( (cj + self.alpha[i][stim[i]])/(nk + self.alpha0[i]) )
        return np.product( pjks )
        
    
    def posterior(self, stim):
        """Find P(k|F) for each cluster"""
        pk = np.zeros( len(self.partition) )
        pFk = np.zeros( len(self.partition) )
        
        # existing clusters:
        for k in range(len(self.partition)):
            pk[k] = self.c * len(self.partition[k])/ ((1-self.c) + self.c * self.N)
            if len(self.partition[k])==0: # case of new cluster
                pk[k] = (1-self.c) / (( 1-self.c ) + self.c * self.N)
            pFk[k] = self.condclusterprob( stim, k)
        
        # put it together
        pkF = (pk*pFk) # / sum( pk*pFk )
        
        return pkF
    
    def stimulate(self, stim):
        """Argmax of P(k|F) + P(0|F)"""
        winner = np.argmax( self.posterior(stim) )
        #print ("Stim: ", stim)
        #print ("Partition: ", self.partition)
        #print (self.posterior(stim))
        
        if len(self.partition[winner]) == 0:
            self.partition.append( [] )
        self.partition[winner].append(stim)
        
        self.N += 1
    
    def query(self, stimulus):
        """Queried value should be -1."""
        qdim = -1
        for i in range(len(stimulus)):
            if stimulus[i] < 0:
                if qdim != -1:
                    raise Exception("ERROR: Multiple dimensions queried.")
                qdim = i
        
        self.N = sum([len(x) for x in self.partition])
        
        pkF = self.posterior(stimulus)
        pkF = pkF[:-1] / sum(pkF[:-1]) # eliminate `new cluster' prob
        
        pjF = np.array( [sum( [ pkF[k] * self.probClustVal(k, qdim, j) \
                for k in range(len(self.partition)-1)] ) 
                for j in range(len( self.alpha[qdim] ))] )
        
        return pjF / sum(pjF)

#def testlocalmapD():
    """
    Tests the Anderson's ratinal model using the Medin & Schaffer (1978) data.
    
    This script will print out the probability that each item belongs to each
    of the existing clusters or to a new cluster, and the model assign it to
    the most likely cluster. To see that the model is working correctly, you
    can follow along with Anderson (1991), which steps through in the same way.
    """
    ''' stims = [[1, 1, 1, 1, 1], # Medin & Schaffer (1978)
             [1, 0, 1, 0, 1], 
             [1, 0, 1, 1, 0],
             [0, 0, 0, 0, 0],
             [0, 1, 0, 1, 1],  
             [0, 1, 0, 0, 0]]'''
    # These are the classic Shepard Type II and Type IV datasets.
    # Uncomment the one you want to try out; you might want to uncomment
    # shuffling the stims too if you don't care about order.
    #stims = [[0, 0, 0, 0], [0, 0, 1, 0], [1, 1, 0, 1], [1, 1, 1, 1], [1, 0, 0, 0], [1, 0, 1, 1], [0, 1, 0, 0], [0, 1, 1, 1]] # Type IV
    #stims = [[0, 0, 0, 0], [0, 0, 1, 0], [1, 1, 0, 0], [1, 1, 1, 0], [1, 0, 0, 1], [1, 0, 1, 1], [0, 1, 0, 1], [0, 1, 1, 1]] # Type II
    
    '''for _ in range(1) :
        model = dLocalMAP([.5, np.ones((len(stims[0]),2))])
        
        shuffle(stims)
        for s in stims:
            model.stimulate(s)
        print(model.partition)'''
        
        #print("Prob vals for 0,0,0,0,?", model.query( [0]*(len(stims[0])-1) + [-1] ))
       

#def main():
    #testlocalmapD()

#if __name__ == '__main__':
    #main()


In [14]:
#creating a list of stimulus
stims=[]
for index, rows in X.iterrows():
    temp_list =[rows.weight, rows.height, rows.label]
    stims.append(temp_list)


The intricacies of RMC are studied from the [paper](https://cseweb.ucsd.edu//~gary/PAPER-SUGGESTIONS/anderson-psych-rev-1991.pdf).

$$P_{i}(j|k)== \frac{c_j + \alpha_j}{n_k + \alpha_0}$$
- Here, $\alpha_{j}$ helps in computing the value of $P_{i}(j|k)$.
- $P_{i}(j|k)$ is the probability of occurance of j in a partition k.
- Higher value of $\alpha_{j}$ for a input feature means more attention to that particular feature.
- The thrid list in $\alpha$ sets the bias among the category labels (1,2,3)


**Training and testing using non-informative prior:** <br>
When the prior is non-informative, all the values in alpha are set to 1.

In [15]:
alpha=[[1]*100, [1]*100, [1]*4]
for _ in range(1) :
    model = dLocalMAP([.5, np.array(alpha,dtype=object)])

In [16]:
shuffle(stims) #Shuffling the training datapoints
for s in stims:
    model.stimulate(s)

Prediction on the test set :

In [17]:
Y_predicted=Y
for i in range(len(Y)) :
    probability=model.query([Y.iloc[i].weight,Y.iloc[i].height,-1])
    result = np.where(probability == np.amax(probability))
    Y_predicted.at[i,'Predicted label']=result[0][0]


In [18]:
Y_predicted

,weight,height,Predicted label
0,74,67,2
1,69,63,2
2,92,81,2
3,64,61,2
4,66,84,2
5,76,68,2
6,61,58,2
7,64,76,2
8,68,66,2
9,34,61,2


**Training and testing considering the mentioned assumptions:** <br>
- As weight is more likely to be used while making the category judgements, alpha for weight(first list in the code) is taken higher than alpha for height(second list in the code).
- As the model is more likely to call someone big as average than big, the third list contains a higher value for category 2 than category 3. 

In [19]:
alpha=[[0.55]*100, [0.45]*100, [0,1,1.5,0.25]] #Here, alpha[2][category] contains the weight for each of the category.
# As there is no category-0, the weight is zero for alpha[2][0]
for _ in range(1) :
    model = dLocalMAP([.5, np.array(alpha,dtype=object)])

In [20]:
shuffle(stims) #Shuffling the training datapoints
for s in stims:
    model.stimulate(s)

Predicting on the test set:

In [21]:
Y_predicted=Y
for i in range(len(Y)) :
    probability=model.query([Y.iloc[i].weight,Y.iloc[i].height,-1])
    result = np.where(probability == np.amax(probability))
    Y_predicted.at[i,'Predicted label']=result[0][0]


In [22]:
Y_predicted

,weight,height,Predicted label
0,74,67,2
1,69,63,2
2,92,81,2
3,64,61,2
4,66,84,2
5,76,68,3
6,61,58,2
7,64,76,2
8,68,66,2
9,34,61,2


The RMC mostly predicts all the labels as 2. In some rare shuffles of the training set it predicts the label as 3 for one of the stimulus y.
- One of the reason behind this can be the provided training dataset is biased. It contains more datapoints corresponding to label 2.

#### Answer 3: Comments on  exchangibility of data for GCM and RMC

**Part 1: Generalized Context Model for categorization**

In [23]:
def gcm(X,Y,category=[1,2,3],alpha=[0.75,0.25],beta=1,gamma=[0.2,0.6,0.2]) :
    exemplars=form_exemplars(X)
    
    Y_predicted=Y
    Y_predicted['Predicted label']=[0]*len(Y)
    for i in range(len(Y)) :
        probability=compute_probability(Y.iloc[i],exemplars,alpha,beta,gamma)
        result = np.where(probability == np.amax(probability))
        #print(Y.iloc[i],result[0][0]+1)
        Y_predicted.at[i,'Predicted label']=result[0][0]+1
    

    return Y_predicted

GCM prediction without shuffling the training set :

In [24]:
predictions1=gcm(X,Y)
predictions1

,weight,height,Predicted label
0,74,67,2
1,69,63,2
2,92,81,3
3,64,61,2
4,66,84,2
5,76,68,3
6,61,58,1
7,64,76,2
8,68,66,2
9,34,61,1


GCM predictions after shuffling the training set :

In [25]:
X_shuffled=X.sample(frac=1)
predictions2=gcm(X_shuffled,Y)
predictions2

,weight,height,Predicted label
0,74,67,2
1,69,63,2
2,92,81,3
3,64,61,2
4,66,84,2
5,76,68,3
6,61,58,1
7,64,76,2
8,68,66,2
9,34,61,1


**Testing on multiple trials:**

In [26]:
for _ in range(10) :
    X_shuffled=X.sample(frac=1)
    predictions2=gcm(X_shuffled,Y)
    print(predictions2['Predicted label'].values)

[2 2 3 2 2 3 1 2 2 1]
[2 2 3 2 2 3 1 2 2 1]
[2 2 3 2 2 3 1 2 2 1]
[2 2 3 2 2 3 1 2 2 1]
[2 2 3 2 2 3 1 2 2 1]
[2 2 3 2 2 3 1 2 2 1]
[2 2 3 2 2 3 1 2 2 1]
[2 2 3 2 2 3 1 2 2 1]
[2 2 3 2 2 3 1 2 2 1]
[2 2 3 2 2 3 1 2 2 1]


**Conclusion: GCM assumes exchangibility of data**

- As it is evident from the above the above trials, the predictions of GCM is independent of the order in which the data appears.
- The GCM model is based on a voting system i.e. counting total votes cast for category R by exemplars divided by total votes casted. The category with maximum probability is assigned as the label. As, during testing for a stimulus the probability for it being in a particular category does't change, the labels also stay the same.

**Part 2: Rational Model of categorization**

In [27]:
stims=[]
for index, rows in X.iterrows():
    temp_list =[rows.weight, rows.height, rows.label]
    stims.append(temp_list)

Prediction without shuffling the dataset :

In [28]:
alpha=[[0.6]*100, [0.3]*100, [0,2,1,0.25]]
for _ in range(1) :
    model = dLocalMAP([.5, np.array(alpha,dtype=object)])

#shuffle(stims) #Shuffling the training datapoints
for s in stims:
    model.stimulate(s)

Y_predicted=Y
for i in range(len(Y)) :
    probability=model.query([Y.iloc[i].weight,Y.iloc[i].height,-1])
    result = np.where(probability == np.amax(probability))
    Y_predicted.at[i,'Predicted label']=result[0][0]
    

In [29]:
Y_predicted

,weight,height,Predicted label
0,74,67,2
1,69,63,2
2,92,81,2
3,64,61,2
4,66,84,2
5,76,68,2
6,61,58,2
7,64,76,2
8,68,66,2
9,34,61,2


Prediction after shuffling the dataset:

In [30]:
alpha=[[0.6]*100, [0.3]*100, [0,2,1,0.25]]
for _ in range(1) :
    model = dLocalMAP([.5, np.array(alpha,dtype=object)])

shuffle(stims) #Shuffling the training datapoints
for s in stims:
    model.stimulate(s)

Y_predicted=Y
for i in range(len(Y)) :
    probability=model.query([Y.iloc[i].weight,Y.iloc[i].height,-1])
    result = np.where(probability == np.amax(probability))
    Y_predicted.at[i,'Predicted label']=result[0][0]

In [31]:
Y_predicted

,weight,height,Predicted label
0,74,67,2
1,69,63,2
2,92,81,2
3,64,61,2
4,66,84,2
5,76,68,2
6,61,58,2
7,64,76,2
8,68,66,2
9,34,61,2


**Testing on Multiple trials:**

In [32]:
for _ in range(10) :
    alpha=[[0.6]*100, [0.3]*100, [0,2,1,0.25]]
    for i in range(1) :
        model = dLocalMAP([.5, np.array(alpha,dtype=object)])

    shuffle(stims) #Shuffling the training datapoints
    for s in stims:
        model.stimulate(s)

    Y_predicted=Y
    for i in range(len(Y)) :
        probability=model.query([Y.iloc[i].weight,Y.iloc[i].height,-1])
        result = np.where(probability == np.amax(probability))
        Y_predicted.at[i,'Predicted label']=result[0][0]
    
    print(Y_predicted['Predicted label'].values)

[2 2 2 2 2 2 2 2 2 2]
[2 2 2 2 2 2 2 2 2 2]
[2 2 2 2 2 2 2 2 2 2]
[2 2 2 2 2 3 2 2 2 2]
[2 2 2 2 2 3 2 2 2 2]
[2 2 2 2 2 3 2 2 2 2]
[2 2 2 2 2 2 2 2 2 2]
[2 2 2 2 2 2 2 2 2 2]
[2 2 2 2 2 2 2 2 2 2]
[2 2 2 2 2 2 2 2 2 2]


**Conclusion: RMC doesnot assume exchangibility of data.**<br>
- The predictions in test set change with shuffling of training data. One of the reason behind this can be the dependence of prediction on the order of partition formation. 
- The [paper](https://cseweb.ucsd.edu//~gary/PAPER-SUGGESTIONS/anderson-psych-rev-1991.pdf) mentions order sensitivity of RMC and can be controlled by changing the values of $\alpha$.

**Further thoughts:** <br>
Both the models can become order senstive, if online learning is applied i.e adding the current test data once the label is predicted to the train data and then predicting the next test data after training with the updated dataset. 

**References and Acknoledgements:**
- Course slides on GCM and RMC
- [The adaptive nature of human categorization-John R Anderson](https://cseweb.ucsd.edu//~gary/PAPER-SUGGESTIONS/anderson-psych-rev-1991.pdf)
- Discussions with Debdeep Paul Chaudhuri and Manjyot Singh Nanra on the concepts and parameter tuning were really helpful.